Elasticsearch
# Buscador de documentos

<p style="font-size: large; margin-top: 100px;">César de Pablo Sánchez</p>
<p style="font-size: large">@zdepablo</p>

In [ ]:
## Code from: https://www.reddit.com/r/IPython/comments/34t4m7/lpt_print_json_in_collapsible_format_in_ipython/

import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [ ]:
import requests

In [ ]:
index_options = '''
{ 
  "mappings" : { 
      "serie" : {
        "properties" : {
          "_links" : {
            "properties" : {
              "nextepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string",
                    "index" : "no"
                  }
                }
              },
              "previousepisode" : {
                "properties" : {
                  "href" : {
                    "type" : "string",
                    "index" : "no"
                  }
                }
              },
              "self" : {
                "properties" : {
                  "href" : {
                    "type" : "string",
                    "index" : "no"
                   }
                }
              }
            }
          },
          "externals" : {
            "properties" : {
              "imdb" : {
                "type" : "string",
                "index" : "no"
              },
              "thetvdb" : {
                "type" : "long",
                "index": "no"
              },
              "tvrage" : {
                "type" : "long",
                "index": "no"
              }
            }
          },
          "genres" : {
            "type" : "string",
            "index": "not_analyzed"
          },
          "id" : {
            "type" : "long"
          },
          "image" : {
            "properties" : {
              "medium" : {
                "type" : "string",
                "index": "no"
              },
              "original" : {
                "type" : "string",
                "index": "no"
              }
            }
          },
          "language" : {
            "type" : "string",
            "index": "not_analyzed"
          },
          "name" : {
            "type" : "string"
          },
          "network" : {
            "properties" : {
              "country" : {
                "properties" : {
                  "code" : {
                    "type" : "string",
                    "index": "not_analyzed"
                  },
                  "name" : {
                    "type" : "string"
                  },
                  "timezone" : {
                    "type" : "string",
                    "index": "not_analyzed"
                  }
                }
              },
              "id" : {
                "type" : "long"
              },
              "name" : {
                "type" : "string"
              }
            }
          },
          "premiered" : {
            "type" : "date",
            "format" : "strict_date_optional_time||epoch_millis"
          },
          "rating" : {
            "properties" : {
              "average" : {
                "type" : "double"
              }
            }
          },
          "runtime" : {
            "type" : "long"
          },
          "schedule" : {
            "properties" : {
              "days" : {
                "type" : "string",
                "index": "not_analyzed"
              },
              "time" : {
                "type" : "date",
                "format" : "hour_minute",
                "ignore_malformed": true
              }
            }
          },
          "status" : {
            "type" : "string",
            "index": "not_analyzed"            
          },
          "summary" : {
            "type" : "string",
            "index": "analyzed",
            "analyzer": "english"
          },
          "type" : {
            "type" : "string",
            "index": "not_analyzed"            
          },
          "updated" : {
            "type" : "long"
          },
          "url" : {
            "type" : "string",
            "index": "not_analyzed"            
          },
          "webChannel" : {
            "properties" : {
              "country" : {
                "properties" : {
                  "code" : {
                    "type" : "string",
                    "index": "not_analyzed"
                  },
                  "name" : {
                    "type" : "string"
                  },
                  "timezone" : {
                    "type" : "string",
                    "index": "not_analyzed"
                  }
                }
              },
              "id" : {
                "type" : "long"
              },
              "name" : {
                "type" : "string"
              }
            }
          },
          "weight" : {
            "type" : "long"
          }
        }
      }
    }
  } 
'''

requests.delete('http://localhost:9200/my_tvseries')

requests.delete('http://localhost:9200/tvseries')

r = requests.post('http://localhost:9200/tvseries', data = index_options)
print r.text

In [ ]:
series = ['breaking bad','blindspot','the knick','house of cards', 'orange is the new black',
          'true detective', 'game of thrones',
          'the tudors','isabel', 'versailles', 'los serrano']

for s in series:  
  data = requests.get('http://api.tvmaze.com/singlesearch/shows?q=' + s ) 
  id = data.json()['id']
  response = requests.post('http://localhost:9200/tvseries/serie/' + str(id), data = data)
  print s + " indexed: " + response.text 

## Objetivos

 - Conocer los diferentes tipos de búsqueda y el lenguaje de consulta
 - Comprender como se implementa la relevancia en ES 
 - Entender como usar las opciones de relevancia para optimizar los resultados de búsqueda

## Búsqueda en ES

Soporta diferentes operaciones/tipos de busqueda:
  - Búsqueda estructurada: 
      - Operadores de seleccion: a.k.a *SELECT* 
      - Operadores de filtrado:  a.k.a *WHERE*
  - **Búsqueda de texto completo** - mas potente que *WHERE c LIKE "regexp"* 
  - Agregación - pero diferente a *GROUP BY* 
  - **Ordenación** - si bien el orden suele definirse al indexar y segun el tipo - vs *SORT BY* 
  - Paginación - en contraste con *LIMIT*


## Operadores de búsqueda - QueryDSL

 - **Query context**: “¿Cómo de bien se ajusta el documento a la consulta?” 
 - **Filter contex**: “¿Cumple el documento la consulta? si/no” 
 
 - Parametros de búsqueda (Search DSL) 
    - from:to 
    - sort 
    - timeout
    - search_type
    - min_score
    - etc... 
   

### Query DSL - búsqueda de texto completo

In [ ]:
import requests

payload = """
{
  "query" : {
     "match_all" : { }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

### Query DSL - Enviando cadenas de búsqueda 

In [ ]:
import requests

payload = """
{
  "query" : {
     "query_string" : { "query" : "name:'Breaking Bad'" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

In [ ]:
payload = """
{
    "query" : {
        "match" : {
            "summary" : "New Mexico"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

### QueryDSL - terms

In [ ]:
payload = """
{
  "query" : {
     "term" : { "name": "breaking" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

Probar la misma query en mayusculas - no hay análisis asi que no matchea nada

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-term-query.html

### Query DSL - terms

In [ ]:
payload = """
{
  "query" : {
     "terms" : { "genres": ["Drama", "Crime"] }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

Se pueden usar como query los terminos de otro documento - por ejemplo, los seguidores de un seguidor

### Query DSL - Búsqueda por rangos

In [ ]:
payload = """
{
  "query" : {
    "range" : {
        "rating.average" : {
            "gte" : 9,
            "lte" : 10,
            "boost" : 2.0
        }
    }
}
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

### Query DSL - Búsqueda por rangos

In [ ]:
payload = """
{
  "query" : {
    "range" : {
        "premiered" : {
            "gte" : "now-1y/y",
            "lte" : "now/y"
        }
    }
}
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

### Query DSL - busquedas booleanas

 [todo] operadores booleanos

### Query DSL - busquedas booleanas

In [ ]:
payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "Thriller" }
        },
        "must_not" : {
            "term" : { "summary" : "mexico" }
        }
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())


In [ ]:
payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "Thriller" }
        },
        "must_not" : {
            "match" : { "summary" : "Mexico" }
        },
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())


In [ ]:
payload = """
{
  "query" : {
    "bool" : {
        "must" : {
            "term" : { "genres" : "Thriller" }
        },
        "must_not" : {
            "term" : { "summary" : "mexico" }
        },
        "should" : [
           {"term" : { "genres" : "Mystery" }},
           {"term" : { "genres" : "Crime" }}

           ],
        "minimum_should_match" : 1,
        "boost" : 1.0
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
RenderJSON(r.json())


### Query DSL - búsqueda  de frase 

In [ ]:
payload = """
{
    "query" : {
        "match_phrase" : {
            "_all" : "Walter White"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty&explain', data = payload)
RenderJSON(r.json())


### Query DSL - Búsqueda por matching parcial

In [ ]:
payload = """
{
  "query" : {
    "prefix" : { "name" : "break" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty', data = payload)
RenderJSON(r.json())


- wildcards 
- regexps

Son búsquedas más costosas - en general expanden el número de terminos y requieren recorrer todo el diccionario

### Query DSL - busquedas borrosas

In [ ]:
payload = """
{
  "query" : {
    "fuzzy" : { "summary" : "mejico" }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())


In [ ]:
payload = """
{
  "query" : {
    "fuzzy" : {
        "summary" : {
            "value" :         "mejico",
            "boost" :         1.0,
            "fuzziness" :     1,
            "prefix_length" : 0,
            "max_expansions": 100
        }
    }
    }
}"""


r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())


### Query DSL - Boosting

In [ ]:

payload = """
{
  "query" : {
     "terms" : { 
         "genres": ["Drama", "Crime"],
         "boost" : 2.0
      }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())


### Búsqueda multiindice

In [ ]:
payload = """
{
    "query" : {
        "match" : {
            "_all" : "John"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries,megacorp/serie,employee/_search', data = payload)
RenderJSON(r.json())


### Búsqueda multiindice

In [ ]:
payload = """
{
    "query" : {
        "match" : {
            "_all" : "John"
        }
    }
}
"""

r = requests.get('http://localhost:9200/tvseries*/serie/_search', data = payload)
RenderJSON(r.json())


### Relevancia en Elasticsearch

### Relevancia:  *Practical Scoring Function*
  - recupera documentos usando un modelo booleano 
  - asigna la relevancia usando una formula basada en ideas 
    - TF-ID
    - Modelo de espacio vectorial
    

### Relevancia por defecto


$$ rel(q,d) = qNorm_q \cdot coord_{q,d} \cdot \sum_{t \in q}{tf_{t,d} \cdot idf_t^2 \cdot boost_t \cdot norm_{t,d}}$$

 - $qNorm_q$ : factor de normalización de las consultas - ignorar
 - $coord_{q,d}$ : *coordination factor* - sube la importancia de los documentos que tienen más terminos de la consulta 
 - $boost_t$: *query boost* - Sube la importancia de un determinado término
 - $norm_{t,d}$: Factor de normalizacion del indice - tiene en cuenta la longitud del documento y opcionalmente *index boost*

## Relevancia - Default score 

In [ ]:
import requests

r = requests.get('http://localhost:9200/tvseries/_search?q=summary:New Mexico')
RenderJSON(r.json())

### Explicando la relevancia 

In [ ]:
r = requests.get('http://localhost:9200/tvseries/_search?q=New Mexico&explain')

In [ ]:
RenderJSON(r.json())

[TODO] Explicar bien cada uno de los parámetros

## Modelos de relevancia alternativa (texto) 

Otras medidas de relavancia para documentos
  - Okapi BM 25
  - DFR
  
Se puede elegir una funcion de similitud por campo. sin embargo ¡requiere reindexar!

Medidas de similitud entre cadenas:
  - Fuzzy similarity
  - Indexacion por ngrams


### Cambiando la medida de relevancia de texto

 - Se define y configura en el mapping para cada campo
 - Se pueden generar configuraciones propias con parámetros fijados para reusar
 - Tunear la relevancia (de texto) es generalmente el último de los ajustes 
     - si se justifica en una aplicación típica
     - debe ser un proceso evaluado en función de la aplicación. 

<pre>
PUT /my_index
{
  "mappings": {
    "doc": {
      "properties": {
        "title": {
          "type":       "string",
          "similarity": "BM25",
          "k1": 2.0, 
          "b": 1.0
        },
        "body": {
          "type":       "string",
          "similarity": "default" 
        }
      }
  }
}
</pre>

## Otras medidas de relevancia (estructura) 

- We can take into account other relevance measures
     - Time - recency
     - Location - proximity
     - Other numerical fields
  - Difference with databases: algorithms are adapted to sort and get top k documents. 

In [ ]:
payload = """
{
  "query" : {
     "terms" : { "genres": ["Drama", "Crime"] }
  },
  "sort" : { "rating.average" : "asc" }
  
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

## Definiendo la relevancia a medida

 - random score
 - function score 
 - script score

### Random Score
  - Necesidad de sacar un conjunto de resultados en orden aleatorio

In [ ]:
payload = """
{
  "query": {
    "function_score" : {
    "query" : { 
       "terms" : { "genres": ["Drama", "Crime"] }
     },
      "random_score" : {  }
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search', data = payload)
RenderJSON(r.json())

### Function score - Incluyendo un factor adicional a la relevancia de texto

In [ ]:
payload = """
{
  "query": {
    "function_score" : {
    "query" : { 
       "terms" : { "genres": ["Drama", "Crime"] }
     },
      "field_value_factor" : {
        "field" : "weight"
     }
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?explain', data = payload)
RenderJSON(r.json())

### Function score: estableciendo la relevancia como "frescura" 

In [ ]:
payload = """
{
  "query": {
    "function_score": {
      "functions": [
        {
          "exp": {
            "premiered": { 
              "origin": "now", 
              "offset": "30d",
              "scale": "360d"
            }
          }
        }
      ]
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?explain', data = payload)
RenderJSON(r.json())

![Funciones de relevancia segun valor](https://www.elastic.co/guide/en/elasticsearch/reference/current/images/decay_2d.png)

In [ ]:
payload = """
{
  "query": {
    "function_score": {
      "query" : { 
       "terms" : { "genres": ["Drama", "Crime"] }
       },
      "functions": [
        {
          "exp": {
            "premiered": { 
              "origin": "now", 
              "offset": "30d",
              "scale": "360d"
            }
          }
        }
      ],
      "score_mode": "sum"
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?explain', data = payload)
RenderJSON(r.json())

### Script score - teniendo en cuenta la puntuacion

In [ ]:
payload = """
{
  "query": {
    "function_score" : {
    "query" : { 
       "terms" : { "genres": ["Drama", "Crime"] }
     },
      "script_score" : {
        "script" : "_score * doc['rating.average'].value"
     }
    }
  }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?explain', data = payload)
RenderJSON(r.json())

Requiere que habilitemos los scripts dinámicos en elasticsearch.yml

script.inline: true
script.indexed: true



## Búsqueda multicampo

Motivation: 

  * Different uses: 
    * Match different full text queries in different fields: title and author
    * Order and bool queries impact, boosting may also be used
    
    * Tuning: 
       * dis_max - selecting the score of the best fields
       * tie_breaker
       * multi_match - helper to direct the same query to different fields
       * we can select fields by using regular expressions 
       * cross fields entity search
       
   * best fields 
   * most fields 
   * cross fields 


## Búsqueda multicampo

In [ ]:
payload = """
{
    "query" : {
       "multi_match" : {
         "query":    "New Mexico",
         "type" : "phrase",
         "fields": [ "name", "summary" ] 
       }
   }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty&explain', data = payload)
RenderJSON(r.json())


## Filtrado de resultados (*filter*)

<pre>
  {
    "query": {...},
    "filter": {...}
  }
</pre>
  - *filter* aplica un filtro a los resultados de la búsqueda (*query*) 
  - el filtro elimina los resultados
  - no afecta a la relevancia (*_score*)
  - los resultados se pueden cachear en memoria
  
  

## Filtrando resultados

In [ ]:
payload = """
{
    "query" : {
       "multi_match" : {
         "query":    "New Mexico",
         "fields": [ "name", "summary" ] 
       }
   },
   "filter" : {
       "terms" : { "genres": ["Adventure", "Comedy"] }

   }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty&explain', data = payload)
RenderJSON(r.json())


## *Highlighting* - integración con la UI de búsqueda

  - ES incluye la posibilidad de resaltar los términos de una búsqueda en su contexto (*highlight*)
  - Generalmente es una labor de la UI, pero se puede llevar al motor por: 
     - Consistencia
     - Rendimiento 

     
## *Highlighting*      
  - Permite: 
    - seleccionar campos para resumen
    - seleccionar longitud y numero de fragmentos
    - seleccionar las etiquetas para resaltar
    - configuracion global o por campo
      
  - Proporciona varios algoritmos en función del tipo de indexación del campo  
     - Plain (Lucene) 
     - Fast Vector Highlighter (term vectors) 
     - Postings Highlighter  (offsets) 
     

## Resaltando los terminos de búsqueda en el resumen (*snippet*)

In [ ]:
payload = """
{
    "query" : {
       "multi_match" : {
         "query":    "New Mexico",
         "fields": [ "name", "summary" ] 
       }
   },
   "highlight" : {
      "pre_tags": ["<span class='my-style'>"],
      "post_tags": ["</span>"],      
      "fields" : {
          "summary" : {},
          "name": {}
      }
   }
}
"""

r = requests.get('http://localhost:9200/tvseries/serie/_search?pretty&explain', data = payload)
RenderJSON(r.json())


## Otras características
  - Plantillas de búsqueda (/_search/template) 
  - More Like This - buscar documentos similares a otro
  - Percolator - consulta que se ejecuta sobre cada documento nuevo que se indexa 
  - Routing 

## Conclusiones
  - Un lenguaje flexible de búsqueda que permite búsquedas complejas
  - Diseñar la funcionalidad de búsqueda adecuada a una aplicación: 
  - Llevar al motor de búsqueda, algunas operaciones de la aplicación: highlighting, percolator, recomendacion  

## Conclusiones - Diseño de la busqueda 
  - Diseño del documento
  - Diseño de la consulta
  - Diseño del análisis de los campos
     - Búsqueda 
     - Indexación
  - Diseño de la relevancia textual o medida de similitud
     - Boosting en tiempo de consulta 
     - Boosting de campos 
     - Boosting de indices
  - Diseño de la relevancia combinando otros campos 
     - Boosting de cada componente
